In [ ]:
import numpy as np
import plotly.graph_objects as go
import IPython
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt

#https://nbviewer.org/github/empet/Plotly-plots/blob/master/Plotly-Slice-in-volumetric-data.ipynb

# Simulation avec gstlearn

In [ ]:
model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=0.5,range=50,space=gl.SpaceRN(3))
grid = gl.DbGrid.create(nx = [100,100,100])
gl.simtub(None,dbout=grid,model=model)

In [ ]:
def get_the_slice(x,y,z, surfacecolor):
    return go.Surface(x=x,
                      y=y,
                      z=z,
                      surfacecolor=surfacecolor,
                      coloraxis='coloraxis')



In [ ]:
def get_lims_colors(surfacecolor):# color limits for a slice
    return np.min(surfacecolor), np.max(surfacecolor)


In [ ]:
def return_slice(grid,name,indSlice,pos = 0,usesel=False):
    
    useback = grid.useSel
    grid.useSel = usesel
    res = grid[name].reshape(grid.getNXs())
    nx = grid.getNX(0)
    ny = grid.getNX(1)
    nz = grid.getNX(2)

    rangeX = grid.getExtrema(0,usesel)
    rangeY = grid.getExtrema(1,usesel)
    rangeZ = grid.getExtrema(2,usesel)
    
    if pos == 0 :
        y = grid.getGridAxis(1)
        z = grid.getGridAxis(2)
        y,z = np.meshgrid(y,z)
        x = grid.getGridAxis(0)[indSlice] * np.ones(y.shape)
        surfcolor = res[indSlice,:,:].T
       
        
    if pos == 1 :
        x = grid.getGridAxis(0)
        z = grid.getGridAxis(2)
        x, z = np.meshgrid(x,z)
        y = grid.getGridAxis(1)[indSlice] * np.ones(x.shape)
        surfcolor = res[:,indSlice,:].T

    
    if pos == 2 :
        x = grid.getGridAxis(0)
        y = grid.getGridAxis(1)
        x, y = np.meshgrid(x,y)
        z = grid.getGridAxis(2)[indSlice] * np.ones(x.shape)
        surfcolor = res[:,:,indSlice].T
    
    sliceRes = get_the_slice(x, y, z, surfcolor)
    
    grid.useSel = useback
    
    return sliceRes

In [ ]:
def colorax(vmin, vmax):
    return dict(cmin=vmin,
                cmax=vmax)


In [ ]:
res = grid["Simu"].reshape(grid.getNXs())
sminz, smaxz = get_lims_colors(res)
data = [return_slice(grid,"Simu",12),
        return_slice(grid,"Simu",22),
        return_slice(grid,"Simu",22,1)
       ]
fig1 = go.Figure(data=data)



fig1.update_layout(
         title_text='Slices in volumetric data', 
         title_x=0.5,
         width=700,
         height=700,
         scene_zaxis_range=grid.getExtrema(0,False),
         scene_xaxis_range=grid.getExtrema(1,False),
         scene_yaxis_range=grid.getExtrema(2,False),
         coloraxis=dict(colorscale='BrBG',
                        colorbar_thickness=25,
                        colorbar_len=0.75,
                        **colorax(sminz, smaxz)))            
      
#fig1.show()  

In [ ]:
plt.imshow(res[:,36,:],cmap='BrBG',norm=plt.Normalize(sminz,smaxz))